In [ ]:
import gdown
import os
import pandas as pd
from google.colab import drive
# part 1
import pandas as pd
import numpy as np
import re
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import pickle
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/Datasets/Amazon/train_data.csv", low_memory=False)
title_brand = pd.read_csv("/content/drive/MyDrive/Datasets/Amazon/title_brand.csv", low_memory=False)
test_df = pd.read_csv("/content/drive/MyDrive/Datasets/Amazon/test_data.csv", low_memory=False)